<a href="https://colab.research.google.com/github/vitorqueijo/nuclei_keras/blob/master/nuclei_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pixels's intensity of positive and negative nuclei
## dataset to detect 20x20 pixel nuclei in histological images

Dataset's source: https://www.kaggle.com/zicouc/pixelss-intensity-of-positive-and-negative-nuclei/

by Vítor Queijo, Science and Technology graduation student

In this easy example, we're going to use Keras, a neural network library fast and simple to implement. On this dataset, it's perceptible the similarity with the classical MNIST exercise, one of the first datasets you will use at the beginning of your ML course. So, it's going to be the same implematation with minor changes in the parameters and preprocessing. I hope it's useful in the future to any other grad student (I spent almost 4 hours to understand how to get from DataFrame to Tensor object)

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## First, let's see with pandas what we're dealing with

In [2]:
train_df: pd.DataFrame = pd.read_csv('data.csv')
train_df.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,...,pixel362,pixel363,pixel364,pixel365,pixel366,pixel367,pixel368,pixel369,pixel370,pixel371,pixel372,pixel373,pixel374,pixel375,pixel376,pixel377,pixel378,pixel379,pixel380,pixel381,pixel382,pixel383,pixel384,pixel385,pixel386,pixel387,pixel388,pixel389,pixel390,pixel391,pixel392,pixel393,pixel394,pixel395,pixel396,pixel397,pixel398,pixel399,pixel400,Label
0,84,72,85,104,89,59,47,48,69,88,112,137,146,124,102,102,120,120,102,86,79,70,75,89,86,69,60,64,79,84,94,115,125,106,87,92,120,121,107,92,...,128,92,80,84,74,74,67,69,75,71,62,62,60,66,81,79,83,89,92,116,118,88,80,93,94,90,70,64,67,60,56,60,58,60,72,90,99,114,130,1
1,196,161,153,174,188,206,169,171,149,140,147,177,200,200,190,185,178,162,131,99,181,168,161,164,174,208,182,175,142,148,169,199,213,208,201,202,214,205,185,161,...,142,135,140,138,132,142,137,149,145,145,141,159,174,159,161,155,133,132,146,128,137,132,138,132,119,127,125,132,129,128,120,140,157,142,142,125,113,129,159,0
2,138,157,172,152,140,139,133,151,158,140,131,136,139,134,131,134,148,146,148,157,145,152,156,138,135,137,125,136,151,141,134,133,132,131,133,138,146,149,153,156,...,125,128,134,140,142,138,134,121,130,142,125,136,138,151,142,145,147,144,137,129,127,128,136,145,148,141,134,115,125,138,125,142,146,153,137,131,138,146,153,0
3,125,132,135,137,147,151,136,113,116,142,157,142,122,115,120,123,131,132,131,127,157,150,135,125,135,149,150,139,126,144,153,146,138,137,134,127,124,131,135,131,...,148,144,139,135,135,141,148,162,162,154,139,129,127,129,129,129,132,135,137,139,137,144,156,157,151,155,167,173,171,161,146,139,142,141,136,127,129,132,133,0
4,126,132,134,128,123,121,119,117,110,102,104,113,113,94,76,69,65,57,45,47,140,122,111,118,126,121,110,104,110,102,99,104,99,82,69,66,71,65,52,52,...,76,78,80,71,44,38,56,36,45,58,52,41,60,89,99,117,134,150,156,66,71,82,86,70,43,44,63,36,50,66,60,58,91,123,126,126,143,161,167,1


In [3]:
train_df.shape

(1185, 401)

What's inside in one of them...

In [4]:
train_df.iloc[0,0:].values

array([ 84,  72,  85, 104,  89,  59,  47,  48,  69,  88, 112, 137, 146,
       124, 102, 102, 120, 120, 102,  86,  79,  70,  75,  89,  86,  69,
        60,  64,  79,  84,  94, 115, 125, 106,  87,  92, 120, 121, 107,
        92,  71,  66,  61,  69,  78,  69,  60,  66,  95,  92,  98, 118,
       123,  90,  63,  63,  89,  97, 101,  98,  68,  69,  62,  64,  74,
        65,  49,  49,  79,  84, 103, 129, 128,  80,  39,  35,  42,  58,
        87, 104,  74,  75,  75,  79,  77,  61,  40,  32,  40,  53,  77,
       103, 100,  57,  24,  27,  22,  34,  73, 104,  83,  75,  85,  91,
        69,  46,  33,  22,  31,  40,  49,  58,  53,  27,  15,  27,  27,
        25,  59,  92, 101,  76,  91,  96,  52,  26,  32,  24,  41,  45,
        40,  33,  29,  18,  13,  23,  31,  18,  49,  87, 125,  86,  98,
       101,  41,  19,  36,  33,  35,  43,  39,  31,  33,  27,  19,  21,
        29,  13,  48,  94, 133, 107,  97,  96,  47,  18,  34,  32,  30,
        32,  36,  31,  22,  29,  35,  25,  30,  32,  42, 103, 13

**Uhmm**...it doens't look like an useful data for keras. 

Let's now **reshape** for 20x20 as informed on kaggle description. Since we're going to put a single row as a vector of a picture, we're taking all the 400 columns and reshaping as an 20x20 with numpy's help. the last row are the labels.

A wonderful explanation from the marvelous Stackoverflow:

*Briefly, feature is input; label is output*.

*A feature is one column of the data in your input set. For instance, if you're trying to predict the type of pet someone will choose, your input features might include age, home region, family income, etc. The label is the final choice, such as dog, fish, iguana, rock, etc.*

*Once you've trained your model, you will give it sets of new input containing those features; it will return the predicted "label" (pet type) for that person* 
- by user: Prune - Systems Validation Engineer, Artificial Intelligence Products Group, Intel Corp.

In [0]:
df_x = train_df.iloc[:,:-1].values.reshape(len(train_df),20,20,1)
y = train_df.iloc[:,-1:].values

In our case, we just have two categories: Positive or Negative nucleus; so, num_classes is suposed to be only 2.
Keras already has a function to do it, this is commonly known as 'one hot encoding' categories, since sometimes we have data that we can transform in number just to have an simple output, but it's not like we're going to make any calculation on it, just to make it simple, we're using even if we already have '1 or 0', educational purposes here.

In [0]:
df_y = keras.utils.to_categorical(y, num_classes=2)

In [0]:
df_x = np.array(df_x)
df_y = np.array(df_y)

In [8]:
df_x.shape, df_y.shape

((1185, 20, 20, 1), (1185, 2))

> ...The famous sklearn to help us split our dataset to train our model...







In [0]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=10)

Now, our neural network: I've chosen the hard sigmoid function for activation on convolution because it was the best in outcome on this example, 20 nodes since each time I've set up higher numbers, the loss skyrocketed and compromised the accuracy evidently. Futhermore, sigmoid function for this input is good to sharpen up to the next layer, which I set up to softmax since we're dealing with two outputs: 1 or 0; and I want a better estimated probability.
Also, the loss "categorical_crossentropy" is better for binary outputs. (I guess.)

In [10]:
model = Sequential()
model.add(Convolution2D(32, 3,data_format='channels_last', activation='hard_sigmoid', input_shape=(20,20,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(20))
model.add(Dropout(.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 18, 18, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2592)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                51860     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 42        
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total para

Let's run a single test with one epoch just for science purposes and not because I'm eager to test it...

In [12]:
model.fit(x_train[:100], y_train[:100], validation_data=(x_test[:20], y_test[:20]))

Instructions for updating:
Use tf.cast instead.
Train on 100 samples, validate on 20 samples
Epoch 1/1
100/100 [==============================] - 0s 3ms/step - loss: 2.8495 - acc: 0.5200 - val_loss: 1.6363 - val_acc: 0.6000


In [13]:
model.fit(x_train, y_train,epochs=10, validation_data=(x_test, y_test))

Train on 888 samples, validate on 297 samples
Epoch 1/10
888/888 [==============================] - 0s 412us/step - loss: 1.2265 - acc: 0.6971 - val_loss: 0.1145 - val_acc: 0.9663
Epoch 2/10
888/888 [==============================] - 0s 430us/step - loss: 0.2661 - acc: 0.9088 - val_loss: 0.0967 - val_acc: 0.9697
Epoch 3/10
888/888 [==============================] - 0s 417us/step - loss: 0.1147 - acc: 0.9673 - val_loss: 0.0435 - val_acc: 0.9899
Epoch 4/10
888/888 [==============================] - 0s 407us/step - loss: 0.1164 - acc: 0.9595 - val_loss: 0.0400 - val_acc: 0.9832
Epoch 5/10
888/888 [==============================] - 0s 433us/step - loss: 0.0993 - acc: 0.9628 - val_loss: 0.0333 - val_acc: 0.9933
Epoch 6/10
888/888 [==============================] - 0s 406us/step - loss: 0.0624 - acc: 0.9809 - val_loss: 0.0425 - val_acc: 0.9899
Epoch 7/10
888/888 [==============================] - 0s 409us/step - loss: 0.2064 - acc: 0.9414 - val_loss: 0.0242 - val_acc: 0.9933
Epoch 8/10
888/8

Wow, 99,33% of accuracy, well, I think we've our winner.

In [15]:
print("print('Thank you for coming to my TED talk')")

print('Thank you for coming to my TED talk')
